<a href="https://colab.research.google.com/github/ashuthoshmartindasari/food-not-food/blob/main/gradio_test_food_vision_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradio Food Vision demo

[Gradio](https://gradio.app/) helps demo your machine learning models with a few lines of code.

Let's test it out using a small food recognition model. Similar to the one trained in notebook 03 of the [Zero to Mastery TensorFlow for Deep Learning course](https://dbourke.link/ZTMTFcourse).

To start we'll install Gradio. 

In [ ]:
!pip install gradio

     |████████████████████████████████| 2.1 MB 8.4 MB/s 
     |████████████████████████████████| 206 kB 74.9 MB/s 
     |████████████████████████████████| 1.9 MB 50.1 MB/s 
     |████████████████████████████████| 63 kB 2.7 MB/s 
     |████████████████████████████████| 961 kB 53.1 MB/s 
     |████████████████████████████████| 3.2 MB 41.5 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4709 sha256=ff026a5e269a13a24fc4e1e5dd93abb149951295e505415495c669dbd9be2107
  Stored in directory: /root/.cache/pip/wheels/13/e4/6c/e8059816e86796a597c6e6b0d4c880630f51a1fcfa0befd5e6
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-py3-none-any.whl size=3372 sha256=d9808910d6e1d9352bff1223fa26187fe0a08c120710f9a53934e1e68b64fcff
  Stored in directory: /root/.cache/pip/wheels/28/c0/c4/44687421dab41455be93112bd1b0dee1f3c5a9aa27bee63708
Successfully built ffmpy flask-cachebuster


## Basic usage

In [ ]:
import gradio as gr

def greet(name):
  return "Hello " + name + "!!"

iface = gr.Interface(fn=greet, inputs="text", outputs="text")
iface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://45561.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://45561.gradio.app')

## Food Vision Demo

In [ ]:
# Download data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
!unzip -q 10_food_classes_10_percent.zip

--2021-08-05 00:52:04--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.128, 142.250.141.128, 142.251.2.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   254MB/s    in 0.6s    

2021-08-05 00:52:04 (254 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [ ]:
train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

In [ ]:
# Setup data inputs
import tensorflow as tf
IMG_SIZE = (224, 224)
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 label_mode="categorical",
                                                                 image_size=IMG_SIZE)

test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE)

# Get class names
class_names = train_data.class_names

class_names

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

### Train model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Setup the input shape to our model
input_shape = (224, 224, 3)

# Create a frozen base model
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Create input and output layers
inputs = layers.Input(shape=input_shape, name="input_layer") # create input layer
x = base_model(inputs, training=False) # pass augmented images to base model but keep it in inference mode, so batchnorm layers don't get updated: https://keras.io/guides/transfer_learning/#build-a-model 
x = layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
outputs = layers.Dense(10, activation="softmax", name="output_layer")(x)
model = tf.keras.Model(inputs, outputs)

# Compile
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), # use Adam optimizer with base learning rate
              metrics=["accuracy"])

# Fit the model saving checkpoints every epoch
model.fit(train_data,
          epochs=10,
          validation_data=test_data,
          validation_steps=int(0.25 * len(test_data)))

16711680/16705208 [==============================] - 0s 0us/step
Epoch 1/10
24/24 [==============================] - 41s 265ms/step - loss: 1.8920 - accuracy: 0.4200 - val_loss: 1.3108 - val_accuracy: 0.7319
Epoch 2/10
24/24 [==============================] - 5s 201ms/step - loss: 1.1208 - accuracy: 0.7573 - val_loss: 0.8914 - val_accuracy: 0.8076
Epoch 3/10
24/24 [==============================] - 5s 203ms/step - loss: 0.8105 - accuracy: 0.8120 - val_loss: 0.7384 - val_accuracy: 0.8257
Epoch 4/10
24/24 [==============================] - 5s 198ms/step - loss: 0.6550 - accuracy: 0.8533 - val_loss: 0.6321 - val_accuracy: 0.8454
Epoch 5/10
24/24 [==============================] - 5s 197ms/step - loss: 0.5583 - accuracy: 0.8720 - val_loss: 0.6101 - val_accuracy: 0.8322
Epoch 6/10
24/24 [==============================] - 5s 193ms/step - loss: 0.4878 - accuracy: 0.8960 - val_loss: 0.5530 - val_accuracy: 0.8553
Epoch 7/10
24/24 [==============================] - 5s 193ms/step - loss: 0.4320 -

In [ ]:
# Create function to pass to Gradio interface
def food_vision(img):
  img = tf.image.resize(img, size=(224, 224))
  img = tf.expand_dims(img, axis=0) # make image tensor right size for model
  pred = model.predict(img)[0]
  output_dict = dict(zip(class_names, map(float, pred)))
  return output_dict

In [ ]:
import gradio as gr

gr.Interface(fn=food_vision,
             inputs=gr.inputs.Image(),
             outputs=gr.outputs.Label(num_top_classes=5)).launch(debug=True) # debug=True = show errors in notebook

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://49577.gradio.app
Interface loading below...
